# COGS 118B - Final Project

# Insert title here

## Group members

- Jacob Hanshaw
- Jonathan Ito
- Hiroki Ito
- Rebecca Nguyen

# Abstract 
This section should be short and clearly stated. It should be a single paragraph <200 words.  It should summarize: 
- what your goal/problem is
- what the data used represents 
- the solution/what you did
- major results you came up with (mention how results are measured) 

__NB:__ this final project form is much more report-like than the proposal and the checkpoint. Think in terms of writing a paper with bits of code in the middle to make the plots/tables

Our project focuses on developing a reinforcement learning (RL) agent to play Super Mario Bros using Stable-Retro. The objective is to optimize the agent’s ability to navigate levels efficiently, avoid obstacles, and maximize rewards. The agent receives pixel-based observations and game state data, including Mario’s position, enemy locations, and score, to make informed decisions.
We implemented policy-based and value-based learning approaches, including Monte Carlo, Deep Q-Networks (DQN), and Double DQN (DDQN). Custom reward functions were designed to reinforce beneficial behaviors, and various hyperparameters were tested to enhance learning efficiency.
Performance was evaluated based on key metrics such as distance traveled, level completion efficiency, and total rewards accumulated. Our findings showed that

### Add "major results" section

# Background

The use of artificial intelligence (AI) in gaming has been a subject of extensive research for years, with applications expanding across various industries. While classic games like Super Mario Bros. may appear simple, they involve complex decision-making due to multiple inputs and dynamic elements such as obstacles, enemies, power-ups, lives, and timers. Demonstrating AI’s ability to handle these challenges effectively highlights its potential for navigating dynamic environments.

Among various AI techniques, reinforcement learning (RL) has emerged as one of the most efficient and widely explored methods. Previous research has applied RL to multiple games, including 
Pokémon <a name="Kalose"></a>[<sup>[2]</sup>](#Kalose), 
Super Mario Bros.<a name="Liao"></a>[<sup>[3]</sup>](#Liao), 
and Street Fighter<a name="Huertas"></a>[<sup>[1]</sup>](#Huertas), 
with Q-learning and other RL-based strategies playing a central role. Many studies indicate that Deep Reinforcement Learning with Double Q-Learning (DDQN) tends to be the most effective model for game-based learning tasks. However, we aimed to test how different RL models perform when compared to one another. This study<a name="De-Yu"></a>[<sup>[4]</sup>](#De-Yu) examines the differences in performance between Deep Q-Networks (DQN) and DDQN under various movement strategies, measuring how efficiency and learning outcomes differ between these approaches. By comparing our results with prior research, we aim to validate expected trends and analyze any variations in performance.

While we draw inspiration from existing studies, our approach involves designing a custom RL agent and conducting independent experiments. Specifically, we evaluate how our agent, trained with modified hyperparameters and reward structures, compares to results from prior research. This allows us to assess the effectiveness of our model in learning optimal gameplay strategies.

We chose Super Mario Bros. as our testing environment for several reasons. One key factor is its structured action space and consistency between episodes. Unlike games such as Pokémon or Street Fighter, where AI agents must react to an unpredictable opponent (a "player 2" or CPU-controlled character), Super Mario Bros. offers a more controlled environment where the primary challenge is navigating the level efficiently. Training AI models for games with reactive opponents often requires significantly more computational power and extended training time. Given our resource constraints, we opted to focus on Super Mario Bros. as a more feasible environment for testing reinforcement learning models while still presenting meaningful challenges in sequential decision-making.



# Problem Statement

The problem that our group is attempting to solve is Super Mario Bros. Specifically, we are trying to train an agent (Mario) to be able to run through levels using reinforcement learning. Instead of focusing on a single level for the agent to train on, it will move through each level sequentially. This means that once it has completed level one (reached the flag), it will automatically be placed at the start of level two. This will go on until the episode ends (the agent reaches the maximum number of steps or it dies). Each game state is represented as an array with shape (240, 256, 3). The first two axes are for the shape of the window and the third axis is for color, as the game is in RGB color. This representation of the game allows us to pass each state into our models with ease.

As we will describe later, the agent will have access to a certain number of actions in order to complete its goal. These actions are represented as integers and don't necessarily have to be only one button press. For example, the movement "right and A" can be a singular action represented by some integer (such as 2). We can control which actions the agent has access to, meaning we are able to control the complexity of the action space.

The agent will receive rewards according to different rewards structures as well. The agent can obtain rewards by defeating enemies, progressing through the level, completing the level quickly, getting coins, etc. A combination of these metrics (defined by each rewards structure) is how we measure the success of the agent. Additionally, the gym environment that we use allows us to render each episode, allowing us to see exactly what the agent is doing. So the rewards are not only recorded, they can be seen happening as well.

# Data

For our data set, we will use a package called `gym_super_mario_bros`, which uses `nes-py` as its base rom to run Super Mario Bros. 

## Data Preprocessing

### State Space Creation

For our data, we used the games' pixel frames to represent states. However, the original size of the frames was too large to compute efficiently. To fix this, we used gyms wrappers and a custom SkipFrame wrapper to preprocess after creating the environment. The following wrappers were applied:

**SkipFrame**:

SkipFrame allows the use of an action for one frame and its application to as many as we specify in the parameter. We decided to skip four frames, which saved us from redundancy. Rewards from the skipped states are merged, and we return the cumulative reward and the most recent step info. 

**ResizeObservation**:

This wrapper allows us to resize each frame from the original RGB input shape of (240, 256, 3) to the shape (84, 84, 3). This size reduction will help reduce computational costs later on with our neural networks. 

**GrayScaleObservation**:

Gray scaling our images does a similar thing by reducing the shape of our yet again. This time, the three RGB channels will be reduced to one grayscale channel, making our frame shape now (84, 84, 1).

**FrameStack**:

Frame stacking helps our network see "motion" better. To do this, we will stack four consecutive frames on each other, making our new shape (4, 84, 84, 1). It is important to note that our original skip frame means that each new frame state represents 16 actual game frames due to the effect of both skipping and stacking frames.


### Action Space Creation

 For our action space, `gym_super_mario_bros` comes with three discrete action spaces labeled RIGHT_ONLY, SIMPLE_MOVEMENT, and COMPLEX_MOVEMENT. To use them in our code, we had to use a `nes-py` wrapper called JoypadSpace. We initiated the wrapper with the RIGHT_ONLY and SIMPLE_MOVEMENT for our tests.

`RIGHT_ONLY = [ 
    ['NOOP'],
    ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
]`

`SIMPLE_MOVEMENT = [
    ['NOOP'],
    ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
    ['A'],
    ['left'],
]`

The code above shows that each action space has only 5-7 actions. Reducing our action space will make it easier for our agent to learn, given the more complex state space.

# Proposed Solution

In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Make sure to describe how the solution will be tested.  

If you know details already, describe how (e.g., library used, function calls) you plan to implement the solution in a way that is reproducible.

If it is appropriate to the problem statement, describe a benchmark model<a name="sota"></a>[<sup>[3]</sup>](#sotanote) against which your solution will be compared. 

# DQN Solution

A Deep Q-Network (DQN) is a clear potential solution to our problem. Due to the size of the game, Super Mario Bros has high dimensionality, which neural networks are equipped to handle. DQNs are able to estimate the Q-values for a state-action space over time, proving to be very effective for complex games. The neural network learns how to best approximate the Q-values so that it is not necessary to store all values directly (which would require far too much space). 

### How the DQN Works

The neural network, or the policy network, used in the DQN will be built using PyTorch. Specifically, we will use Convolutional Neural Networks (CNNs) in order to extract the most important features from each frame. We input the frame dimensions into the CNN, which will then feed into a Linear model, which will have an output with the action space dimensions. The activation function we will use is ReLU, and we will be using the Adam optimizer. 

Besides the neural network, there are a few key features of the DQN. First is the Q-function which represents the maximum cumulative reward that the agent can achieve from a state by taking a certain action. There is also a replay buffer that stores past experiences (state, action, reward, next state) that the model samples from during training. This is used to help optimize the model with gradient descent. Importantly, the DQN computes the Q-values using the same network that is being trained. More specifically, it selects an action that maximizes the Q-value and also estimates the future reward using the same policy network.

# DDQN Solution

We use a DDQN (Double Deep Q-Network) and compare it to the original DQN. A DDQN works similarly to a DQN except that it uses two different neural networks to select the best action and then evaluate the result of that action instead of one to do both tasks. This helps to reduce overestimation of the Q-values which should stabilize learning. Our goal for the DDQN is the same as the DQN. We want to fill out our complex action value function with many state-action pairs. As already said, we still use neural networks because of their ability to handle high dimensionality.

### How the DDQN Works

The CNN is still the base of our DDQN solution. CNNs are great at extracting visual features from our frames. Our input to the CNN will be the frame dimensions, and the output will be the action space shape. The value in each of the output neurons represents a predicted q value for that action during that state. 

We will use two networks to accomplish this. The first network is the online network. This network is the one that we will be training. The second is the target network, which contains the ground truth for the predictions in the loss function. The target network will occasionally copy the weights of the online network to increase stability while training. In order to do this, the target network samples a random (state, action, reward, next_state, done) tuple from the replay buffer. This state in the tuple is passed to the online network to get the predicted q values for each action. However, we only focus on the action taken in our original tuple. To check its accuracy, we use the next state in the tuple to get a q value from the target network instead of checking all actions for the best q value. We would then compute the target value using the function r + gamma * max_next_actionQ(next_state, next_action). This value is then passed into the loss function with the predicted Q(s,a) from earlier in the online network. 

- Note about loss and optimizer function: We used the Adam optimizer for our online network and MSE loss as our loss function. 

# Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

# Results

You may have done tons of work on this. Not all of it belongs here. 

Reports should have a __narrative__. Once you've looked through all your results over the quarter, decide on one main point and 2-4 secondary points you want us to understand. Include the detailed code and analysis results of those points only; you should spend more time/code/plots on your main point than the others.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Subsection 1

You will likely have different subsections as you go through your report. For instance you might start with an analysis of the dataset/problem and from there you might be able to draw out the kinds of algorithms that are / aren't appropriate to tackle the solution.  Or something else completely if this isn't the way your project works.

### Subsection 2

Another likely section is if you are doing any feature selection through cross-validation or hand-design/validation of features/transformations of the data

### Subsection 3

Probably you need to describe the base model and demonstrate its performance.  Probably you should include a learning curve to demonstrate how much better the model gets as you increase the number of trials

### Subsection 4

Perhaps some exploration of the model selection (hyper-parameters) or algorithm selection task. Generally reinforement learning tasks may require a huge amount of training, so extensive grid search is unlikely to be possible. However expoloring a few reasonable hyper-parameters may still be possible.  Validation curves, plots showing the variability of perfromance across folds of the cross-validation, etc. If you're doing one, the outcome of the null hypothesis test or parsimony principle check to show how you are selecting the best model.

###  DDQN 

Maybe you do model selection again, but using a different kind of metric than before?  Or you compare a completely different approach/alogirhtm to the problem? Whatever, this stuff is just serving suggestions.

### Reward Functions

After running our initial DDQN model, which seemed to be performing the best, we wanted to look at how different rewward functions may impact the success of the agent. In the initial reward function utilized 3 components: _x_reward, _time_penalty, and _death_peanalty. It gave rewards if the agent moved foward and took points when it moved back, remove 1 point every time a tick occurs to encourage speed, and a death penalty that took points away when the agent dies.

We wanted to atttempt 3 different reward functions to compare with the inital model that was given: RewardMoveRight, RewardScoreBased, and RewardAntiDeath. 


 
**RewardMoveRight**: Encourage riskier bejacior by removing death peantly, and encourage the agent to move right. When moving right, it is award points and docked if moving left or is stationary. There was also a time peanlaty, docking and awarding points based on the time left. 

The agent after 
as there was no death penatly, it began to run through the stage qukcly but would periodically get stuck on certian obstacles. When a certain time threshold would hit, the reward function begins punishing the agent. This was to encourage the model to run throuhg the stage quicker to avoid it, but it led to the agent immideiatly dying to prevent losing points. This is likely due to the lack of 

Good at killing the goombas for some reaosn?? 



#RewardAntiDeath function was intended to promote safe exploring, and avoid death at all costs, by giving it a heavy penalty when it dies, and awarding reward points depending on the number of lives remaining at the end of the iteration, and the amount of time left. However, we ran into many issues with this code, particularly surrounding accessing the _is_dying and _is_dead function to identify the status of the agent. When attempting to access the function , 

 lr=0.00025, 
                 gamma=0.9, 
                 epsilon=1.0, 
                 eps_decay=0.99999975, 
                 eps_min=0.1, 
                 replay_buffer_capacity=100_000, 
                 batch_size=32, 
                 sync_network_rate=10000

Future could have been different loss function

# Discussion

### Interpreting the result

OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.


### Limitations

Are there any problems with the work?  For instance would more data change the nature of the problem? Would it be good to explore more hyperparams than you had time for?   

1. Computational power
2. access to the main gym env



### Future work
Looking at the limitations and/or the toughest parts of the problem and/or the situations where the algorithm(s) did the worst... is there something you'd like to try to make these better.
1. time constraints, more iteratoins on reward function to mitigate death issues and 
2. use a vm model to expidite the training


### Ethics & Privacy

If your project has obvious potential concerns with ethics or data privacy discuss that here.  Almost every ML project put into production can have ethical implications if you use your imagination. Use your imagination.

Even if you can't come up with an obvious ethical concern that should be addressed, you should know that a large number of ML projects that go into producation have unintended consequences and ethical problems once in production. How will your team address these issues?

Consider a tool to help you address the potential issues such as https://deon.drivendata.org


### Conclusion

Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.

# Footnotes
<a name="Huertas"></a>1.[^](#Huertas): Huertas, Zambrano, and Diaz Salamanca. Deep Reinforcement Learning for Optimal Gameplay in Street Fighter III: A Resource-Constrained Approach, repositorio.uniandes.edu.co/entities/publication/cb08b574-f75e-4d50-9c43-469099ec6795. <br> 
<a name="Kalose"></a>2.[^](#Kalose): Kalose, Akshay, et al. “Optimal Battle Strategy in Pokémon Using Reinforcement ...” Optimal Battle Strategy in Pokemon Using Reinforcement Learning , web.stanford.edu/class/aa228/reports/2018/final151.pdf. Accessed 15 Feb. 2025.  <br> 
<a name="Liao"></a>3.[^](#Lias): Liao, Yizheng, et al. CS229 Final Report Reinforcement Learning to Play Mario, cs229.stanford.edu/proj2012/LiaoYiYang-RLtoPlayMario.pdf. Accessed 15 Feb. 2025.  <br> 

<a name="De-Yu"></a>4.[^](#De-Yu): Yu, De. Playing Super Mario Bros with Deep Reinforcement Learning, https://www.analyticsvidhya.com/blog/2021/06/playing-super-mario-bros-with-deep-reinforcement-learning
Accessed 15 Feb. 2025. </br>

@misc{gym-super-mario-bros,
  author = {Christian Kauten},
  howpublished = {GitHub},
  title = {{S}uper {M}ario {B}ros for {O}pen{AI} {G}ym},
  URL = {https://github.com/Kautenja/gym-super-mario-bros},
  year = {2018},
}

Kauten, C. (2018). Super Mario Bros for OpenAI Gym. GitHub. Retrieved from https://github.com/Kautenja/gym-super-mario-bros.